In [ ]:
import joblib
import pandas as pd
import numpy as np

# Load and examine the scaler
scaler = joblib.load('scaler.joblib')

print("=== SCALER DEBUG ===")
print(f"Scaler type: {type(scaler)}")
print(f"Feature names (if available): {getattr(scaler, 'feature_names_in_', 'Not available')}")
print(f"Scaler mean_: {scaler.mean_}")
print(f"Scaler scale_: {scaler.scale_}")
print(f"Scaler var_: {scaler.var_}")

# Test with your input values
test_data = pd.DataFrame({
    'Age': [40.0],
    'BMI': [35.16]
})

print("\n=== ORIGINAL VALUES ===")
print(test_data)

print("\n=== SCALED VALUES ===")
scaled = scaler.transform(test_data)
print(f"Scaled Age: {scaled[0][0]}")
print(f"Scaled BMI: {scaled[0][1]}")

print("\n=== EXPECTED REASONABLE SCALED VALUES ===")
print("Age 40 should scale to roughly -1 to +2 range")
print("BMI 35.16 should scale to roughly -2 to +3 range")

# Let's see what the reverse transform gives us
print("\n=== REVERSE TRANSFORM TEST ===")
try:
    original = scaler.inverse_transform(scaled)
    print(f"Reverse transformed: Age={original[0][0]:.2f}, BMI={original[0][1]:.2f}")
except Exception as e:
    print(f"Error in reverse transform: {e}")

# Check if scaler was trained on reasonable data
if hasattr(scaler, 'mean_') and len(scaler.mean_) >= 2:
    print(f"\n=== TRAINING DATA STATISTICS (from scaler) ===")
    print(f"Age - Mean: {scaler.mean_[0]:.2f}, Std: {np.sqrt(scaler.var_[0]):.2f}")
    if len(scaler.mean_) > 1:
        print(f"BMI - Mean: {scaler.mean_[1]:.2f}, Std: {np.sqrt(scaler.var_[1]):.2f}")

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import joblib

# --- Load the Trained Model ---
MODEL_FILE = 'diabetes_model.keras'

def load_model_only():
    """Load only the model, skip the problematic scaler."""
    if not os.path.exists(MODEL_FILE):
        print(f"Error: Model file not found.")
        return None

    print(f"Loading model from '{MODEL_FILE}'...")
    model = tf.keras.models.load_model(MODEL_FILE)
    print("Model loaded successfully.")
    return model

def interactive_prediction_no_scaling(model):
    """Interactive prediction WITHOUT using the corrupted scaler."""
    if model is None:
        return

    print("\n--- Interactive Diabetes Risk Prediction (No Scaling) ---")
    print("Please enter the following details:")

    try:
        # --- Collect User Input ---
        age = float(input("Enter Age (e.g., 35): "))
        gender_str = input("Enter Gender ('female' or 'male'): ").lower().strip()
        gender = 0 if gender_str == 'female' else 1

        history_str = input("Family history of diabetes? ('yes' or 'no'): ").lower().strip()
        family_history = 1 if history_str == 'yes' else 0

        bp_str = input("Have you ever had high blood pressure? ('yes' or 'no'): ").lower().strip()
        high_bp = 1 if bp_str == 'yes' else 0

        exercise_str = input("Enter exercise frequency ('never', '1-2 times', '3-4 times', '5 or more times'): ").lower().strip()
        exercise_map = {
            'never': 0,
            '1-2 times': 1,
            '3-4 times': 2,
            '5 or more times': 3
        }
        exercise_freq = exercise_map.get(exercise_str, 0)

        weight_kg = float(input("Enter Weight in Kilograms (e.g., 70): "))
        height_cm = float(input("Enter Height in Centimeters (e.g., 175): "))

        # --- Calculate BMI ---
        height_m = height_cm / 100
        if height_m <= 0:
            print("Error: Height must be greater than zero.")
            return

        bmi = weight_kg / (height_m ** 2)

        # --- Manual scaling with reasonable values ---
        # Assuming typical ranges: Age 18-80, BMI 15-50
        age_scaled = (age - 45) / 15  # Rough standardization
        bmi_scaled = (bmi - 25) / 8   # Rough standardization

        print(f"\n--- Debug: Manual Scaling ---")
        print(f"Original Age: {age}, Scaled: {age_scaled:.3f}")
        print(f"Original BMI: {bmi:.2f}, Scaled: {bmi_scaled:.3f}")

        # --- Create input array ---
        input_array = np.array([[age_scaled, gender, family_history, high_bp, exercise_freq, bmi_scaled]], dtype=np.float32)

        print(f"Final input: {input_array}")

        # --- Make Prediction ---
        prediction_prob = model.predict(input_array, verbose=0)[0][0]

        print(f"\n--- Debug: Raw Prediction ---")
        print(f"Raw probability: {prediction_prob}")

        # --- Determine risk level ---
        prediction_class = "High Risk" if prediction_prob > 0.5 else "Low Risk"

        # --- Display the Result ---
        print("\n--- Prediction Result ---")
        print(f"The model predicts a '{prediction_class}' of diabetes.")
        print(f"Confidence Score (Probability): {prediction_prob:.2%}")
        print("-------------------------\n")

    except ValueError as e:
        print(f"\nError: Invalid input - {e}")
    except Exception as e:
        print(f"\nAn unexpected error occurred: {e}")
        import traceback
        traceback.print_exc()

# --- Test with different scaling approaches ---
def test_multiple_scaling_approaches(model):
    """Test the same input with different scaling methods."""
    # Test data: Age=40, Gender=1, Family_History=1, High_BP=1, Exercise=0, BMI=35.16

    print("\n=== TESTING DIFFERENT SCALING APPROACHES ===")

    # Approach 1: No scaling
    input1 = np.array([[40, 1, 1, 1, 0, 35.16]], dtype=np.float32)
    pred1 = model.predict(input1, verbose=0)[0][0]
    print(f"No scaling: {pred1:.4f}")

    # Approach 2: Light scaling
    input2 = np.array([[(40-45)/15, 1, 1, 1, 0, (35.16-25)/8]], dtype=np.float32)
    pred2 = model.predict(input2, verbose=0)[0][0]
    print(f"Light scaling: {pred2:.4f}")

    # Approach 3: Z-score with typical population stats
    input3 = np.array([[(40-50)/20, 1, 1, 1, 0, (35.16-27)/6]], dtype=np.float32)
    pred3 = model.predict(input3, verbose=0)[0][0]
    print(f"Population Z-score: {pred3:.4f}")

    # Approach 4: Min-max scaling (assuming Age 18-80, BMI 15-50)
    input4 = np.array([[(40-18)/(80-18), 1, 1, 1, 0, (35.16-15)/(50-15)]], dtype=np.float32)
    pred4 = model.predict(input4, verbose=0)[0][0]
    print(f"Min-max scaling: {pred4:.4f}")

# --- Main execution ---
if __name__ == '__main__':
    loaded_model = load_model_only()
    if loaded_model:


        # Test different approaches
        test_multiple_scaling_approaches(loaded_model)

        # Interactive session
        while True:
            interactive_prediction_no_scaling(loaded_model)
            another = input("Do you want to make another prediction? (yes/no): ").lower().strip()
            if another != 'yes':
                print("Exiting interactive session.")
                break